In [1]:
#Group Project 8 - Heart Diseease
#IDS 400
#11/25/2020


In [1]:
#imports
import pandas as pd
import numpy as np

#list of imports check over this 
from sklearn.ensemble import RandomForestClassifier #for the model
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz #plot tree
from sklearn.metrics import roc_curve, auc #for model evaluation
from sklearn.metrics import classification_report #for model evaluation
from sklearn.metrics import confusion_matrix #for model evaluation
from sklearn.model_selection import train_test_split #for data splitting
# import eli5 #for purmutation importance
# from eli5.sklearn import PermutationImportance
import shap #for SHAP values
from pdpbox import pdp, info_plots #for partial plots
np.random.seed(123) #ensure reproducibility

pd.options.mode.chained_assignment = None 

ModuleNotFoundError: No module named 'shap'

In [13]:
#data import and formatting
heart = pd.read_csv('heart.csv')

#change column names
heart.columns = ['age', 'sex', 'chest_pain_type', 'resting_blood_pressure', 'cholesterol', 'fasting_blood_sugar', 'rest_ecg', 'max_heart_rate_achieved','exercise_induced_angina', 'st_depression', 'st_slope', 'num_major_vessels', 'thalassemia', 'target']

#     age: The person's age in years
#     sex: The person's sex (1 = male, 0 = female)
#     cp: The chest pain experienced (Value 1: typical angina, Value 2: atypical angina, Value 3: non-anginal pain, Value 4: asymptomatic)
#     trestbps: The person's resting blood pressure (mm Hg on admission to the hospital)
#     chol: The person's cholesterol measurement in mg/dl
#     fbs: The person's fasting blood sugar (> 120 mg/dl, 1 = true; 0 = false)
#     restecg: Resting electrocardiographic measurement (0 = normal, 1 = having ST-T wave abnormality, 2 = showing probable or definite left ventricular hypertrophy by Estes' criteria)
#     thalach: The person's maximum heart rate achieved
#     exang: Exercise induced angina (1 = yes; 0 = no)
#     oldpeak: ST depression induced by exercise relative to rest ('ST' relates to positions on the ECG plot. See more here)
#     slope: the slope of the peak exercise ST segment (Value 1: upsloping, Value 2: flat, Value 3: downsloping)
#     ca: The number of major vessels (0-3)
#     thal: A blood disorder called thalassemia (3 = normal; 6 = fixed defect; 7 = reversable defect)
#     target: Heart disease (0 = no, 1 = yes)

heart.head(10)

,age,sex,chest_pain_type,resting_blood_pressure,cholesterol,fasting_blood_sugar,rest_ecg,max_heart_rate_achieved,exercise_induced_angina,st_depression,st_slope,num_major_vessels,thalassemia,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
5,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
6,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1
7,44,1,1,120,263,0,1,173,0,0.0,2,0,3,1
8,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1
9,57,1,2,150,168,0,1,174,0,1.6,2,0,2,1


In [17]:
#program readability
heart['sex'][heart['sex'] == 0] = 'female'
heart['sex'][heart['sex'] == 1] = 'male'

#categorical types
heart['chest_pain_type'][heart['chest_pain_type'] == 1] = 'typical angina'
heart['chest_pain_type'][heart['chest_pain_type'] == 2] = 'atypical angina'
heart['chest_pain_type'][heart['chest_pain_type'] == 3] = 'non-anginal pain'
heart['chest_pain_type'][heart['chest_pain_type'] == 4] = 'asymptomatic'

heart['fasting_blood_sugar'][heart['fasting_blood_sugar'] == 0] = 'lower than 120mg/ml'
heart['fasting_blood_sugar'][heart['fasting_blood_sugar'] == 1] = 'greater than 120mg/ml'

heart['rest_ecg'][heart['rest_ecg'] == 0] = 'normal'
heart['rest_ecg'][heart['rest_ecg'] == 1] = 'ST-T wave abnormality'
heart['rest_ecg'][heart['rest_ecg'] == 2] = 'left ventricular hypertrophy'

heart['exercise_induced_angina'][heart['exercise_induced_angina'] == 0] = 'no'
heart['exercise_induced_angina'][heart['exercise_induced_angina'] == 1] = 'yes'

heart['st_slope'][heart['st_slope'] == 1] = 'upsloping'
heart['st_slope'][heart['st_slope'] == 2] = 'flat'
heart['st_slope'][heart['st_slope'] == 3] = 'downsloping'

heart['thalassemia'][heart['thalassemia'] == 1] = 'normal'
heart['thalassemia'][heart['thalassemia'] == 2] = 'fixed defect'
heart['thalassemia'][heart['thalassemia'] == 3] = 'reversable defect'

heart.head()

,age,sex,chest_pain_type,resting_blood_pressure,cholesterol,fasting_blood_sugar,rest_ecg,max_heart_rate_achieved,exercise_induced_angina,st_depression,st_slope,num_major_vessels,thalassemia,target
0,63,male,non-anginal pain,145,233,greater than 120mg/ml,normal,150,no,2.3,0,0,normal,1
1,37,male,atypical angina,130,250,lower than 120mg/ml,ST-T wave abnormality,187,no,3.5,0,0,fixed defect,1
2,41,female,typical angina,130,204,lower than 120mg/ml,normal,172,no,1.4,flat,0,fixed defect,1
3,56,male,typical angina,120,236,lower than 120mg/ml,ST-T wave abnormality,178,no,0.8,flat,0,fixed defect,1
4,57,female,0,120,354,lower than 120mg/ml,ST-T wave abnormality,163,yes,0.6,flat,0,fixed defect,1


In [20]:
heart['sex'] = heart['sex'].astype('object')
heart['chest_pain_type'] = heart['chest_pain_type'].astype('object')
heart['fasting_blood_sugar'] = heart['fasting_blood_sugar'].astype('object')
heart['rest_ecg'] = heart['rest_ecg'].astype('object')
heart['exercise_induced_angina'] = heart['exercise_induced_angina'].astype('object')
heart['st_slope'] = heart['st_slope'].astype('object')
heart['thalassemia'] = heart['thalassemia'].astype('object')

heart.head()

,age,sex,chest_pain_type,resting_blood_pressure,cholesterol,fasting_blood_sugar,rest_ecg,max_heart_rate_achieved,exercise_induced_angina,st_depression,st_slope,num_major_vessels,thalassemia,target
0,63,male,non-anginal pain,145,233,greater than 120mg/ml,normal,150,no,2.3,0,0,normal,1
1,37,male,atypical angina,130,250,lower than 120mg/ml,ST-T wave abnormality,187,no,3.5,0,0,fixed defect,1
2,41,female,typical angina,130,204,lower than 120mg/ml,normal,172,no,1.4,flat,0,fixed defect,1
3,56,male,typical angina,120,236,lower than 120mg/ml,ST-T wave abnormality,178,no,0.8,flat,0,fixed defect,1
4,57,female,0,120,354,lower than 120mg/ml,ST-T wave abnormality,163,yes,0.6,flat,0,fixed defect,1


In [22]:
heart = pd.get_dummies(heart, drop_first=True)
heart.head()



,age,resting_blood_pressure,cholesterol,max_heart_rate_achieved,st_depression,num_major_vessels,target,sex_male,chest_pain_type_atypical angina,chest_pain_type_non-anginal pain,chest_pain_type_typical angina,fasting_blood_sugar_lower than 120mg/ml,rest_ecg_left ventricular hypertrophy,rest_ecg_normal,exercise_induced_angina_yes,st_slope_flat,st_slope_upsloping,thalassemia_fixed defect,thalassemia_normal,thalassemia_reversable defect
0,63,145,233,150,2.3,0,1,1,0,1,0,0,0,1,0,0,0,0,1,0
1,37,130,250,187,3.5,0,1,1,1,0,0,1,0,0,0,0,0,1,0,0
2,41,130,204,172,1.4,0,1,0,0,0,1,1,0,1,0,1,0,1,0,0
3,56,120,236,178,0.8,0,1,1,0,0,1,1,0,0,0,1,0,1,0,0
4,57,120,354,163,0.6,0,1,0,0,0,0,1,0,0,1,1,0,1,0,0


In [27]:

#seperates training and validation, x_test is validation, random_state is key
X_train, X_test, y_train, y_test = train_test_split(heart.drop('target', 1), heart['target'], test_size = .2, random_state=10) #split the data

model = RandomForestClassifier(max_depth=5)
model.fit(X_train, y_train)

#gini, decision tree classifier

RandomForestClassifier(max_depth=5)

In [31]:
y_predict = model.predict(X_test)
y_pred_quant = model.predict_proba(X_test)[:, 1]
y_pred_bin = model.predict(X_test)

print(y_predict,'\n',y_pred_quant,'\n', y_pred_bin)

[0 0 0 1 0 1 1 1 1 0 1 0 1 1 0 1 0 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 0 0 0 0 1
 1 1 0 0 0 1 0 1 0 0 1 1 0 1 1 0 0 1 1 1 1 0 0 1] 
 [0.09308107 0.37045738 0.24984389 0.81507667 0.23917373 0.71667277
 0.68360618 0.71906586 0.90870823 0.13179535 0.92598897 0.18038518
 0.70256888 0.81298941 0.28354345 0.7594792  0.21402179 0.0284612
 0.55094194 0.10246333 0.12976264 0.82451312 0.65033067 0.86414143
 0.41803762 0.07136725 0.03009109 0.21826602 0.84664023 0.03801267
 0.85604423 0.12122709 0.22196684 0.30055853 0.0136618  0.11106504
 0.61989277 0.52296152 0.77670096 0.27779114 0.05401185 0.34099817
 0.8024916  0.44267071 0.88848795 0.48313123 0.44368278 0.93728387
 0.71912733 0.35356078 0.53439951 0.79557328 0.04295178 0.25489437
 0.94798941 0.85222624 0.78541988 0.81336203 0.01440706 0.07288636
 0.73665527] 
 [0 0 0 1 0 1 1 1 1 0 1 0 1 1 0 1 0 0 1 0 0 1 1 1 0 0 0 0 1 0 1 0 0 0 0 0 1
 1 1 0 0 0 1 0 1 0 0 1 1 0 1 1 0 0 1 1 1 1 0 0 1]


In [29]:
confusion_matrix = confusion_matrix(y_test, y_pred_bin)
confusion_matrix

array([[28,  7],
       [ 4, 22]], dtype=int64)